In [1]:
import os
import random
import shutil

In [4]:
base_dir = "C:/Users/DeepScan/Downloads/Images/test_data/"
real_dest_dir = base_dir + 'train_real_faces'
fake_dest_dir = base_dir + 'train_fake_faces'

real_src_dir = "C:/Users/DeepScan/Downloads/Images/working_data_with_48_real_fps/train_real_faces"
fake_src_dir = "C:/Users/DeepScan/Downloads/Images/working_data_with_48_real_fps/train_fake_faces"

In [5]:
def move_files(src, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)

    src_files = os.listdir(src)
    for i in range(15):
        random.shuffle(src_files)

    for file in src_files[:5000]:
        shutil.copy2(os.path.join(src, file), dest)

move_files(real_src_dir, real_dest_dir)
move_files(fake_src_dir, fake_dest_dir)

## Individual Models Evaluation

This evaluation is being performed using the sota weights. sota here refers to the best model trained by us on smaller subset like 400 real & fake videos & now wanna test out on large quantity of unseen test data.

In [19]:
# File Processing
import os
import random
import shutil
from datetime import datetime

# Face extraction, cropping & display
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Model Training
from __future__ import print_function
from itertools import chain
import csv
import random
import os.path as osp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.transforms import ToPILImage
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm
import torchvision.transforms as transforms
import timm
from tabulate import tabulate

from sklearn.metrics import log_loss, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

import dlib


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
base_dir = "C:/Users/DeepScan/Downloads/Images/test_data/"
train_real_faces = base_dir + 'train_real_faces'
train_fake_faces = base_dir + 'train_fake_faces'

train_real_data = [os.path.join(train_real_faces, f) for f in os.listdir(train_real_faces)]
train_fake_data = [os.path.join(train_fake_faces, f) for f in os.listdir(train_fake_faces)]

test_images = train_fake_data + train_real_data

In [24]:
for i in range(15):
    np.random.shuffle(test_images)

frame = pd.DataFrame(test_images, columns=['path'])
# Add the new 'label' column
frame['label'] = frame['path'].apply(lambda x: 0 if 'real' in x else 1)
frame.to_csv(f'{base_dir}/celebdf_v2_test_images.csv', index=False, sep=',', header=True, encoding='utf-8')

celebdf_test_csv = pd.read_csv(f"{base_dir}/celebdf_v2_test_images.csv")

In [25]:
PATH = "Z:/Deepscan-Repository/evaluation/sota_weights"
model_name = "swin_base_patch4_window7_224"

for file in os.listdir(PATH):
    if file.endswith(".pth"):
        model_path = os.path.join(PATH, file)
        model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2).to(device)
        model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))

        print(f'+++++++++++++Evaluating {model_name} trained on CelebDF-v2+++++++++++++')
        print(f"+++++++++++++Loaded {file} weights!+++++++++++++")

        # Create the save path
        save_dir = os.path.join(f"Z:/Deepscan-Repository/evaluation/swin_no_aug_testsplit_randomframes/400_400_subset_sota/{file}")
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        save_path = os.path.join(save_dir, f"{os.path.splitext(file)[0]}_pth_result.csv")

        torch.manual_seed(42)
        Evaluate(model, celebdf_test_csv, save_path)

        print('====================================================================================================\n\n')

+++++++++++++Evaluating swin_base_patch4_window7_224 trained on CelebDF-v2+++++++++++++
+++++++++++++Loaded 10_epochs.pth weights!+++++++++++++


Processing images: 100%|██████████| 313/313 [00:29<00:00, 10.44batch/s]


+-----------+-------------+----------+------------+
|   Logloss |   AUC Score |       F1 |   Accuracy |
+===========+=============+==========+============+
|   3.08927 |    0.248183 | 0.303041 |     0.2116 |
+-----------+-------------+----------+------------+


+++++++++++++Evaluating swin_base_patch4_window7_224 trained on CelebDF-v2+++++++++++++
+++++++++++++Loaded 12_epochs.pth weights!+++++++++++++


Processing images: 100%|██████████| 313/313 [00:23<00:00, 13.42batch/s]


+-----------+-------------+----------+------------+
|   Logloss |   AUC Score |       F1 |   Accuracy |
+===========+=============+==========+============+
|   3.61621 |    0.248503 | 0.314326 |     0.2208 |
+-----------+-------------+----------+------------+


+++++++++++++Evaluating swin_base_patch4_window7_224 trained on CelebDF-v2+++++++++++++
+++++++++++++Loaded 15_epochs.pth weights!+++++++++++++


...

+-----------+-------------+----------+------------+
|   Logloss |   AUC Score |       F1 |   Accuracy |
+===========+=============+==========+============+
|   3.82732 |    0.143728 | 0.565076 |     0.3975 |
+-----------+-------------+----------+------------+


+++++++++++++Evaluating swin_base_patch4_window7_224 trained on CelebDF-v2+++++++++++++
+++++++++++++Loaded 3_epochs.pth weights!+++++++++++++


Processing images: 100%|██████████| 313/313 [00:28<00:00, 11.06batch/s]


+-----------+-------------+----------+------------+
|   Logloss |   AUC Score |       F1 |   Accuracy |
+===========+=============+==========+============+
|   1.78937 |    0.229448 | 0.374643 |     0.2552 |
+-----------+-------------+----------+------------+


+++++++++++++Evaluating swin_base_patch4_window7_224 trained on CelebDF-v2+++++++++++++
+++++++++++++Loaded 7_epochs.pth weights!+++++++++++++


Processing images: 100%|██████████| 313/313 [00:28<00:00, 10.97batch/s]


+-----------+-------------+----------+------------+
|   Logloss |   AUC Score |       F1 |   Accuracy |
+===========+=============+==========+============+
|   2.52889 |    0.271209 | 0.295834 |     0.2107 |
+-----------+-------------+----------+------------+




In [11]:
from tqdm import tqdm

def get_scores(predictions):
    true_labels = []
    predicted_labels = []
    predicted_probs = []

    for key, value in predictions.items():
        predicted_probs.append(value[0])
        predicted_labels.append(value[1])
        true_labels.append(value[2])

    logloss = log_loss(true_labels, predicted_probs)
    auc = roc_auc_score(true_labels, predicted_probs)
    f1 = f1_score(true_labels, predicted_labels)
    acc = accuracy_score(true_labels, predicted_labels)
        
    print(tabulate([[logloss, auc, f1, acc]], headers=['Logloss', 'AUC Score', 'F1', 'Accuracy'], tablefmt="grid"))
    return logloss, auc, f1, acc

def Evaluate(model, faces_path_dict, save_path, batch_size=32):
        
    labels_map = ['real', 'fake']
    predictions = {}
    faces = []

    # Preprocess the faces
    for img in faces_path_dict['path']:
        if img.endswith('.jpg'):
            face = cv2.imread(img)
            faces.append(face)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    # Create batches of preprocessed face tensors
    face_tensors = [transform(Image.fromarray(face)).unsqueeze(0).to(device) for face in faces]
    batches = [face_tensors[i:i+batch_size] for i in range(0, len(face_tensors), batch_size)]

    # Run the model on the batches
    model.eval()
    with torch.no_grad():
        for batch in tqdm(batches, desc="Processing images", unit="batch"):
            outputs = model(torch.cat(batch, dim=0))
            batch_probability_fake = torch.softmax(outputs, 1)[:, 1].tolist()
            batch_probability_real = torch.softmax(outputs, 1)[:, 0].tolist()

            for i, img in enumerate(faces_path_dict['path'][len(predictions):len(predictions)+len(batch)]):
                original_label = 0 if 'real' in img else 1
                if labels_map[torch.argmax(outputs[i]).item()] == 'fake':
                    predictions[img] = [batch_probability_fake[i], 1, original_label]
                else:
                    predictions[img] = [batch_probability_real[i], 0, original_label]

                # Print the tabulated data for each iteration
                # print(tabulate([[img, predictions[img][0], predictions[img][1], predictions[img][2]]], tablefmt='simple_grid'))

    with open(save_path, "w", newline="") as csvfile:
        fieldnames = ["ImagePath", "ProbabilityScore", "PredictedLabel", "OriginalLabel"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, lineterminator="\n")
        writer.writeheader()
        for img, prediction in predictions.items():
            writer.writerow(
                {
                    "ImagePath": img,
                    "ProbabilityScore": prediction[0],
                    "PredictedLabel": prediction[1],
                    "OriginalLabel": prediction[2],
                }
            )

    logloss, auc, f1, acc = get_scores(predictions)
    with open(save_path, "a", newline="") as csvfile:
        fieldnames = ["LogLoss", "AUC", "F1-Score", "Accuracy"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, lineterminator="\n")
        writer.writeheader()
        writer.writerow({"LogLoss": logloss, "AUC": auc, "F1-Score": f1, "Accuracy": acc})

    torch.cuda.empty_cache()
